In [500]:
import pandas as pd
import numpy as np
import os
from datetime import timedelta, date

'''This script produces a file which shows availability. Each row is a booking
date, and each col is a (room type, stay date) tuple'''

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# load raw hotel data
df_data = pd.read_csv('data/2018_clean.csv', parse_dates=['ARRIVAL', 'DEPART', 'LOOK_DATE'])
df_data = (df_data.sort_values(['LOOK_DATE', 'RESNO'])
           .drop(['RESNO'], axis=1))

df_caps = pd.read_csv('data/capacities.csv', index_col='UNIT')

In [501]:
# helper vars
look_start = df_data['LOOK_DATE'].min()
look_end = df_data['LOOK_DATE'].max()
look_range = pd.date_range(look_start, look_end)

ssn_start = df_data['ARRIVAL'].min()
ssn_end = df_data['ARRIVAL'].max()
ssn_range = pd.date_range(ssn_start, ssn_end)

# get max periods per day from transactions script
periods_per_day = (df_data[df_data['CANCEL_INDICATOR'] == 0] # look at bookings only
                   .drop(['CANCEL_INDICATOR'], axis=1)
                   .groupby('group_id').first() # collapse groups
                   .sort_values('LOOK_DATE')
                   .groupby(by=['LOOK_DATE', 'ARRIVAL']).count()
                   .max()[0])
intraday_range = range(0, periods_per_day)


In [502]:
# initializing blank df with same height as transactions

# generate room type list
df_grouped_types = df_data.groupby(by='UNIT').count()
unit_list = df_grouped_types.index.tolist()

# create blank df
df_avail = pd.DataFrame(index=pd.MultiIndex.from_product([look_range, ssn_range, intraday_range], 
                                                           names=['LOOK_DATE', 'ARRIVAL', 'INTRADAY']),
                        columns=unit_list)
df_avail = df_avail.fillna(0.0)

# add default capacities
for index, row in df_capa.iterrows():
    df_avail[index] = df_avail[index] + row['CAPACITY']

In [503]:
df_avail

2BV  4BV    CD    CK    DD   DK   DKB
LOOK_DATE  ARRIVAL    INTRADAY                                       
2018-01-02 2018-05-11 0         8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      1         8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      2         8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      3         8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      4         8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      5         8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      6         8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      7         8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      8         8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      9         8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      10        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      11        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      12        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      13        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      14        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      15        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      16        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      17        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      18        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      19        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      20        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      21        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      22        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      23        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      24        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      25        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      26        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      27        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      28        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      29        8.0  4.0  13.0  17.0  24.0  6.0  12.0
...                             ...  ...   ...   ...   ...  ...   ...
2018-10-27 2018-10-27 40        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      41        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      42        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      43        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      44        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      45        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      46        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      47        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      48        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      49        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      50        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      51        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      52        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      53        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      54        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      55        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      56        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      57        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      58        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      59        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      60        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      61        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      62        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      63        8.0  4.0  13.0  17.0  24.0  6.0  12.0
                      64        8.0  4.0  

In [504]:
row = df_data.iloc[0,:]
df_test = df_avail.copy()
cur_idx = (row['LOOK_DATE'], row['ARRIVAL'], 1)
cur_idx2 = (row['LOOK_DATE'], row['ARRIVAL'])
df_test.loc[cur_idx2, row['UNIT']]

INTRADAY
0     4.0
1     4.0
2     4.0
3     4.0
4     4.0
5     4.0
6     4.0
7     4.0
8     4.0
9     4.0
10    4.0
11    4.0
12    4.0
13    4.0
14    4.0
15    4.0
16    4.0
17    4.0
18    4.0
19    4.0
20    4.0
21    4.0
22    4.0
23    4.0
24    4.0
25    4.0
26    4.0
27    4.0
28    4.0
29    4.0
     ... 
40    4.0
41    4.0
42    4.0
43    4.0
44    4.0
45    4.0
46    4.0
47    4.0
48    4.0
49    4.0
50    4.0
51    4.0
52    4.0
53    4.0
54    4.0
55    4.0
56    4.0
57    4.0
58    4.0
59    4.0
60    4.0
61    4.0
62    4.0
63    4.0
64    4.0
65    4.0
66    4.0
67    4.0
68    4.0
69    4.0
Name: 4BV, Length: 70, dtype: float64

In [505]:
# fill in capacities by iterating over transaction data

for index, row in df_data.iterrows():
    print(row['LOOK_DATE'])
    book_date = row['LOOK_DATE']
    arrive_date = row['ARRIVAL']
    cancel_tf = row['CANCEL_INDICATOR']
    cap_change = 1/row['grp_size']
    print(cap_change)

    # list current cells in avail. that match book and arrival date
    cur_idx = (row['LOOK_DATE'], row['ARRIVAL'])

    # subtract or add capacity for each day in stay dates
    if row['CANCEL_INDICATOR'] == 0:
        # loop subtract (i know this is bad practice lol)
        for i in intraday_range:
            cur_idx = (row['LOOK_DATE'], row['ARRIVAL'], i)
            df_avail.loc[cur_idx, row['UNIT']] -= cap_change
    else:
        for i in intraday_range:
            cur_idx = (row['LOOK_DATE'], row['ARRIVAL'], i)
            df_avail.loc[cur_idx, row['UNIT']] += cap_change

2018-01-02 00:00:00
0.2
2018-01-02 00:00:00
0.25
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
0.2
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
0.25
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
0.3333333333333333
2018-01-02 00:00:00
0.3333333333333333
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
0.25
2018-01-02 00:00:00
0.25
2018-01-02 00:00:00
0.25
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
1.0
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
1.0
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
0.5
2018-01-02 00:00:00
0.3333333333333333
2018-01-02 00:00:00
0.3333333333333333
2018-01-03 00:00:00
0.5
2018-01-03 00:00:00
0.5
2018-01-03 00:00:00
0.3333333333333333
2018-01-03 00:00:00
0.3333333333333333
2018-01-03 00:00:00
0.5
2018-01-03 00:00:00
1.0
2018-01-03 00:00:

2018-01-13 00:00:00
0.5
2018-01-13 00:00:00
0.5
2018-01-13 00:00:00
0.5
2018-01-13 00:00:00
0.5
2018-01-13 00:00:00
0.5
2018-01-13 00:00:00
0.5
2018-01-13 00:00:00
0.5
2018-01-13 00:00:00
0.5
2018-01-13 00:00:00
0.5
2018-01-13 00:00:00
0.5
2018-01-13 00:00:00
0.5
2018-01-13 00:00:00
0.5
2018-01-13 00:00:00
0.5
2018-01-13 00:00:00
0.25
2018-01-13 00:00:00
0.25
2018-01-13 00:00:00
0.25
2018-01-13 00:00:00
0.25
2018-01-14 00:00:00
0.5
2018-01-14 00:00:00
0.5
2018-01-14 00:00:00
0.5
2018-01-14 00:00:00
0.5
2018-01-14 00:00:00
1.0
2018-01-14 00:00:00
1.0
2018-01-14 00:00:00
0.25
2018-01-14 00:00:00
1.0
2018-01-14 00:00:00
0.25
2018-01-14 00:00:00
1.0
2018-01-14 00:00:00
1.0
2018-01-14 00:00:00
0.5
2018-01-14 00:00:00
0.25
2018-01-14 00:00:00
1.0
2018-01-14 00:00:00
1.0
2018-01-14 00:00:00
1.0
2018-01-14 00:00:00
1.0
2018-01-15 00:00:00
1.0
2018-01-15 00:00:00
0.5
2018-01-15 00:00:00
0.5
2018-01-15 00:00:00
0.5
2018-01-15 00:00:00
0.3333333333333333
2018-01-15 00:00:00
0.3333333333333333
201

2018-01-22 00:00:00
0.5
2018-01-22 00:00:00
0.5
2018-01-23 00:00:00
1.0
2018-01-23 00:00:00
1.0
2018-01-23 00:00:00
1.0
2018-01-23 00:00:00
1.0
2018-01-23 00:00:00
1.0
2018-01-23 00:00:00
1.0
2018-01-23 00:00:00
0.5
2018-01-23 00:00:00
0.3333333333333333
2018-01-23 00:00:00
0.5
2018-01-23 00:00:00
0.5
2018-01-23 00:00:00
0.5
2018-01-23 00:00:00
0.5
2018-01-23 00:00:00
1.0
2018-01-23 00:00:00
0.3333333333333333
2018-01-23 00:00:00
1.0
2018-01-23 00:00:00
1.0
2018-01-23 00:00:00
0.5
2018-01-23 00:00:00
1.0
2018-01-23 00:00:00
0.2
2018-01-23 00:00:00
0.5
2018-01-23 00:00:00
0.25
2018-01-23 00:00:00
0.3333333333333333
2018-01-23 00:00:00
1.0
2018-01-23 00:00:00
0.2
2018-01-23 00:00:00
0.5
2018-01-23 00:00:00
0.25
2018-01-23 00:00:00
1.0
2018-01-23 00:00:00
1.0
2018-01-23 00:00:00
0.3333333333333333
2018-01-23 00:00:00
0.5
2018-01-23 00:00:00
1.0
2018-01-23 00:00:00
0.5
2018-01-23 00:00:00
0.5
2018-01-23 00:00:00
1.0
2018-01-23 00:00:00
0.3333333333333333
2018-01-23 00:00:00
1.0
2018-01-23 

2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
0.5
2018-02-01 00:00:00
1.0
2018-02-01 00:00:00
0.5
2018-02-01 00:00:00
0.5
2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
0.2
2018-02-01 00:00:00
0.2
2018-02-01 00:00:00
0.2
2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
0.5
2018-02-01 00:00:00
0.5
2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
0.5
2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
0.5
2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
0.5
2018-02-01 00:00:00
0.5
2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
0.5
2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
0.25
2018-02-01 00:00:00
1.0
2018-02-01 00:00:00
0.5
2018-02-01 00:00:00
1.0
2018-02-01 00:00:00
1.0
2018-02-01 00:00:00
1.0
2018-02-01 00:00:00
0.5
2018-02-01 00:00:00
1.0
2018-02-02 00:00:00
1.0
2018-02-02 00:00:00
1.0
2018-02-02 00:00:00
1.0

2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
1.0
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
1.0
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
1.0
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.25
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
1.0
2018-02-08 00:00:00
0.5
2018-02-08 00:00:00
0.5
2018-02-09 00:00:00
0.5
2018-02-09 00:00:00
1.0
2018-02-09 00:00:00
0.5
2018-02-09 00:00:00
0.5
2018-02-09 00:00:00
1.0
2018-02-09 00:00:00
0.5
2018-02-09 00:00:00
0.5
2018-02-09 00:00:00
0.25
2018-02-09 00:00:00
0.3333333333333333
2018-02-09 00:00:00
0.5
2018-02-09 00:00:00
0.2
2018-02-09 00:00:00
0.5
2018-02-09 00:00:00
0.3

...

2018-09-16 00:00:00
1.0
2018-09-16 00:00:00
1.0
2018-09-16 00:00:00
1.0
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
1.0
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
1.0
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
1.0
2018-09-16 00:00:00
1.0
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
1.0
2018-09-16 00:00:00
0.5
2018-09-16 00:00:00
0.5
2018-09-16 00:00

2018-09-19 00:00:00
1.0
2018-09-19 00:00:00
0.5
2018-09-19 00:00:00
0.16666666666666666
2018-09-19 00:00:00
1.0
2018-09-19 00:00:00
1.0
2018-09-19 00:00:00
1.0
2018-09-19 00:00:00
1.0
2018-09-19 00:00:00
1.0
2018-09-19 00:00:00
0.5
2018-09-19 00:00:00
0.5
2018-09-19 00:00:00
0.5
2018-09-19 00:00:00
0.5
2018-09-19 00:00:00
0.5
2018-09-19 00:00:00
0.5
2018-09-19 00:00:00
0.5
2018-09-20 00:00:00
1.0
2018-09-20 00:00:00
1.0
2018-09-20 00:00:00
1.0
2018-09-20 00:00:00
1.0
2018-09-20 00:00:00
0.3333333333333333
2018-09-20 00:00:00
0.25
2018-09-20 00:00:00
0.2
2018-09-20 00:00:00
0.3333333333333333
2018-09-20 00:00:00
0.5
2018-09-20 00:00:00
0.5
2018-09-20 00:00:00
0.5
2018-09-20 00:00:00
0.3333333333333333
2018-09-20 00:00:00
0.5
2018-09-20 00:00:00
0.3333333333333333
2018-09-20 00:00:00
0.3333333333333333
2018-09-20 00:00:00
0.5
2018-09-20 00:00:00
0.25
2018-09-20 00:00:00
0.5
2018-09-20 00:00:00
0.3333333333333333
2018-09-20 00:00:00
0.3333333333333333
2018-09-20 00:00:00
0.5
2018-09-20 00

2018-09-27 00:00:00
0.14285714285714285
2018-09-27 00:00:00
0.14285714285714285
2018-09-27 00:00:00
1.0
2018-09-27 00:00:00
1.0
2018-09-27 00:00:00
1.0
2018-09-27 00:00:00
1.0
2018-09-27 00:00:00
1.0
2018-09-27 00:00:00
1.0
2018-09-27 00:00:00
0.25
2018-09-27 00:00:00
0.25
2018-09-27 00:00:00
0.25
2018-09-27 00:00:00
0.25
2018-09-27 00:00:00
0.5
2018-09-27 00:00:00
0.3333333333333333
2018-09-27 00:00:00
0.3333333333333333
2018-09-27 00:00:00
0.3333333333333333
2018-09-27 00:00:00
1.0
2018-09-28 00:00:00
0.5
2018-09-28 00:00:00
1.0
2018-09-28 00:00:00
0.25
2018-09-28 00:00:00
1.0
2018-09-28 00:00:00
1.0
2018-09-28 00:00:00
1.0
2018-09-28 00:00:00
1.0
2018-09-28 00:00:00
0.25
2018-09-28 00:00:00
0.25
2018-09-28 00:00:00
0.25
2018-09-28 00:00:00
0.25
2018-09-28 00:00:00
1.0
2018-09-28 00:00:00
1.0
2018-09-28 00:00:00
1.0
2018-09-28 00:00:00
1.0
2018-09-28 00:00:00
0.16666666666666666
2018-09-28 00:00:00
0.16666666666666666
2018-09-28 00:00:00
0.16666666666666666
2018-09-28 00:00:00
0.1666

2018-10-05 00:00:00
0.5
2018-10-05 00:00:00
1.0
2018-10-05 00:00:00
0.5
2018-10-05 00:00:00
0.5
2018-10-05 00:00:00
0.5
2018-10-05 00:00:00
0.5
2018-10-05 00:00:00
1.0
2018-10-05 00:00:00
0.3333333333333333
2018-10-05 00:00:00
0.5
2018-10-05 00:00:00
0.3333333333333333
2018-10-05 00:00:00
0.5
2018-10-05 00:00:00
0.3333333333333333
2018-10-05 00:00:00
1.0
2018-10-05 00:00:00
0.3333333333333333
2018-10-05 00:00:00
1.0
2018-10-05 00:00:00
0.5
2018-10-05 00:00:00
0.5
2018-10-05 00:00:00
1.0
2018-10-05 00:00:00
1.0
2018-10-05 00:00:00
1.0
2018-10-05 00:00:00
1.0
2018-10-05 00:00:00
1.0
2018-10-06 00:00:00
0.5
2018-10-06 00:00:00
0.5
2018-10-06 00:00:00
0.5
2018-10-06 00:00:00
0.5
2018-10-06 00:00:00
0.5
2018-10-06 00:00:00
1.0
2018-10-06 00:00:00
0.5
2018-10-06 00:00:00
0.5
2018-10-06 00:00:00
1.0
2018-10-06 00:00:00
0.5
2018-10-06 00:00:00
0.5
2018-10-06 00:00:00
0.5
2018-10-06 00:00:00
0.5
2018-10-06 00:00:00
0.5
2018-10-06 00:00:00
0.5
2018-10-06 00:00:00
0.5
2018-10-06 00:00:00
0.5
2018

2018-10-19 00:00:00
0.5
2018-10-19 00:00:00
1.0
2018-10-19 00:00:00
0.5
2018-10-19 00:00:00
1.0
2018-10-19 00:00:00
1.0
2018-10-19 00:00:00
0.5
2018-10-19 00:00:00
0.5
2018-10-19 00:00:00
0.5
2018-10-19 00:00:00
0.5
2018-10-19 00:00:00
0.5
2018-10-19 00:00:00
0.5
2018-10-19 00:00:00
0.5
2018-10-19 00:00:00
0.5
2018-10-20 00:00:00
1.0
2018-10-20 00:00:00
1.0
2018-10-21 00:00:00
0.25
2018-10-21 00:00:00
0.3333333333333333
2018-10-21 00:00:00
0.3333333333333333
2018-10-21 00:00:00
0.3333333333333333
2018-10-21 00:00:00
1.0
2018-10-21 00:00:00
0.25
2018-10-21 00:00:00
0.25
2018-10-21 00:00:00
1.0
2018-10-22 00:00:00
0.3333333333333333
2018-10-22 00:00:00
0.3333333333333333
2018-10-22 00:00:00
0.3333333333333333
2018-10-22 00:00:00
0.25
2018-10-22 00:00:00
0.25
2018-10-22 00:00:00
0.5
2018-10-22 00:00:00
0.5
2018-10-22 00:00:00
0.5
2018-10-22 00:00:00
1.0
2018-10-22 00:00:00
1.0
2018-10-23 00:00:00
0.5
2018-10-23 00:00:00
0.25
2018-10-23 00:00:00
0.3333333333333333
2018-10-23 00:00:00
1.0
2

In [519]:
df_avail.to_csv('data/availability.csv')